# BTC´s volatility (APIs)

# Setup

In [1]:
# Secrets keys
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("CHARTSTUDIO_API_KEY")
secret_value_1 = user_secrets.get_secret("FRED_API_KEY")
secret_value_2 = user_secrets.get_secret("NASDAQ_API_KEY")

In [2]:
pip install coinmetrics-api-client

  Attempting uninstall: typer
    Found existing installation: typer 0.4.2
    Uninstalling typer-0.4.2:
      Successfully uninstalled typer-0.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.3.1 requires typer<0.5.0,>=0.3.0, but you have typer 0.6.1 which is incompatible.
spacy 3.3.1 requires typing-extensions<4.2.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.4.0 which is incompatible.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install coinmetrics-api-client -U

Note: you may need to restart the kernel to use updated packages.


# BTC

In [4]:
from os import environ
import sys
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import seaborn as sns
from coinmetrics.api_client import CoinMetricsClient
import logging

In [5]:
sns.set_theme()
sns.set(rc={'figure.figsize':(12,8)})
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [6]:
# We recommend privately storing your API key in your local environment.
try:
    api_key = environ["CM_API_KEY"]
    logging.info("Using API key found in environment")
except KeyError:
    api_key = ""
    logging.info("API key not found. Using community client")


client = CoinMetricsClient(api_key)

assets = ['btc', 'eth']
asset_mapping = {i: assets[i] for i in range(len(assets))}
print(asset_mapping)

import pandas as pd
from coinmetrics.api_client import CoinMetricsClient
api_key = '<api_key>'
client = CoinMetricsClient()

{0: 'btc', 1: 'eth'}


In [7]:
end_time = datetime.now().strftime('%Y-%m-%d')
asset_metrics = client.get_asset_metrics(
    assets='btc', 
    metrics=['PriceUSD'], 
    start_time='2011-01-01T00:00:00Z', 
    end_time=end_time
)
btc_api = asset_metrics.to_dataframe()

In [8]:
btc_api['time'] = btc_api['time'].dt.strftime("%Y-%m-%d")
btc_api = btc_api [['time', 'PriceUSD']]
btc_api.columns = ['time', 'btc_p']

# Traditional assets

In [9]:
pip install fredapi

Note: you may need to restart the kernel to use updated packages.


In [10]:
from fredapi import Fred
fred = Fred(api_key=secret_value_1)

In [11]:
dxy_api = fred.get_series('DTWEXBGS') 
dxy_api = dxy_api.to_frame()
dxy_api.reset_index(inplace=True)
dxy_api.columns = ['time','dxy_p']
dxy_api['time'] = dxy_api['time'].dt.strftime("%Y-%m-%d")
dxy_api['dxy_p'] = dxy_api['dxy_p'].interpolate()

In [12]:
sp500_api = fred.get_series('SP500')
sp500_api = sp500_api.to_frame()
sp500_api.reset_index(inplace=True)
sp500_api.columns = ['time','sp500_p']
sp500_api['time'] = sp500_api['time'].dt.strftime("%Y-%m-%d")
sp500_api['sp500_p'] = sp500_api['sp500_p'].interpolate()

In [13]:
pip install Nasdaq-Data-Link

Note: you may need to restart the kernel to use updated packages.


In [14]:
import nasdaqdatalink
nasdaqdatalink.read_key(filename=secret_value_2)

In [15]:
gold_api = nasdaqdatalink.get("LBMA/GOLD") 

In [16]:
gold_api.reset_index(inplace=True)
gold_api.columns = ['time', 'gold_p', '2', '3', '4', '5', '6']
gold_api = gold_api[['time', 'gold_p']]
gold_api['time'] = gold_api['time'].dt.strftime("%Y-%m-%d")
gold_api['gold_p'] = gold_api['gold_p'].interpolate()

In [17]:
btc = btc_api
dxy = dxy_api
sp500 = sp500_api
gold = gold_api

# Volatility

In [18]:
# returns 
btc['btc_roi'] = np.log(btc.btc_p) - np.log(btc.btc_p.shift(1))
dxy['dxy_roi'] = np.log(dxy.dxy_p) - np.log(dxy.dxy_p.shift(1))
sp500['sp500_roi'] = np.log(sp500.sp500_p) - np.log(sp500.sp500_p.shift(1))
gold['gold_roi'] = np.log(gold.gold_p) - np.log(gold.gold_p.shift(1))

df = pd.merge(pd.merge(sp500,btc,on='time'),dxy,on='time')
df = pd.merge(gold,df,on='time')

roi = df[['time', 'btc_roi',
          'dxy_roi',
          'sp500_roi',
          'gold_roi']]

In [19]:
# Volatility

windows = 30
df['btc_vol'] = roi['btc_roi'].rolling(windows).std()*np.sqrt(windows)
df['dxy_vol'] = roi['dxy_roi'].rolling(windows).std()*np.sqrt(windows)
df['sp500_vol'] = roi['sp500_roi'].rolling(windows).std()*np.sqrt(windows)
df['gold_vol'] = roi['gold_roi'].rolling(windows).std()*np.sqrt(windows)

vol = df[['time', 'btc_vol',
          'dxy_vol',
          'sp500_vol',
          'gold_vol']]

# Online graph

In [20]:
pip install chart_studio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 745.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
from matplotlib.pyplot import plot
import plotly.express as px
import plotly.graph_objects as go

In [22]:
fig = px.line(vol, x="time", y=["btc_vol", 'dxy_vol', 'sp500_vol', 'gold_vol'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [23]:
import chart_studio

In [24]:
import chart_studio.plotly as py
chart_studio.tools.set_credentials_file(username='emigiupponi', 
                                        api_key=secret_value_0)

In [25]:
py.plot(fig, filename = 'vol', auto_open=True)
fig.show()